ELEC-E8105 

Non-linear filtering and parameter estimation Spring 2019


Exercise round 4

# Exercise 1. (Extended Kalman Filter)

Consider the following non-linear state space model

$$ x_k = x_{k-1} - 0.01 sin (x_{k-1}) + q_{k_1}$$

$$y_k = 0.5 sin (2x_k) + r_k$$

where $q_{k-1}$ has variance $0.01^2$ and $r_k$ has variance $0.02$.

**(a)** Derive the required derivatives for EKF for this model and check the
derivatives numerically (recall that $df(x)/dx \approx (f(x+h)-f(x))/h$ when
$h$ is small).


$\frac{df(x)}{dx} = \frac{d( x - 0.01 sin (x))}{dx} = 1 -0.01 cos(x) $

$\frac{dh(x)}{dx} = \frac{d(0.5sin(2x)}{dx} = 0.5 cos(2x)*2 = cos(2x) $

In [ ]:
import numpy
import math
from sklearn.metrics import mean_squared_error

In [ ]:
def f(x):
    return x -0.01*math.sin(x)
def h(x):
    return 0.5*math.sin(2*x)
    

In [ ]:
def df(x):
    return 1-0.01*math.cos(x)
def dh(x):
    return math.cos(2*x)

In [ ]:
x = 1
dx = 0.01

In [ ]:
(f(x+dx) -f(x))/dx

In [ ]:
df(x)

In [ ]:
(h(x+dx)-h(x))/dx

In [ ]:
dh(x)

**(b)** Implement the EKF for the model. Simulate trajectories from the model,
compute the RMSE values and plot the results.

In [ ]:
import scipy.io as sio
from filterpy.kalman import ExtendedKalmanFilter
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from numpy.random import normal
import seaborn as sns
%matplotlib notebook

In [ ]:
np.random.seed(123)
n = 100
q_sd = 0.01
#I assume here that in exercise there was mistake as sqrt(0.02)=0.14 which is very high...
r_sd = 0.02
x_meas = np.zeros(n)
y_meas = np.zeros(n)
x_true = np.zeros(n)
y_true = np.zeros(n)
x_true[0] = 0
y_true[0] = h(x_true[0])
for j in range(1,n):
    x_meas[j] = f(x_true[j-1]) + normal(0,q_sd)
    y_true[j] = h(x_meas[j])
    #I assume here that 
    y_meas[j] = y_true[j] + normal(0,r_sd)

In [ ]:
plt.figure(figsize=(20,10))
t = np.linspace(0,n,n)
plt.title("Trajectories")
plt.plot(t,y_true,label = "True signal",linestyle='--',color='b',alpha=0.5)
plt.scatter(t,y_meas,label = "Measurements",s=80, facecolors='none', edgecolors='g',alpha=0.5)
#    plt.ylim(-0.04,0.04)
plt.ylabel('y')
plt.xlabel('timestep')
plt.legend()
plt.show()

In [ ]:
#RMSE
np.sqrt(mean_squared_error(y_true,y_meas))

# Extended kalman filter

In [ ]:
filterPy = False

In [ ]:
if not filterPy:
    est_y = np.zeros(n)
    mk = np.array([f(0)])
    Pk = np.array([0.01**2])
    for k in range(n):
        #Prediction
        mk_ =  np.array([[f(mk)]])
        F = np.array([[df(mk)]])
        Pk_ = F @ np.array([Pk])@ F.T + q_sd**2
        #Update
        v_k = np.array([[y_meas[k] - h(mk_)]])
        H = np.array([[dh(mk_)]])
        S_k = H@Pk_@H.T + r_sd**2
        K_k = Pk_@H.T @ np.linalg.inv(S_k)
        mk = (mk_ + K_k @ v_k).flatten()
        Pk = (Pk_ - K_k @S_k@K_k).flatten()

        est_y[k] = mk

In [ ]:

plt.figure(figsize=(20,10))
plt.suptitle("Trajectories and EKF estimates")
plt.plot(t,y_true,label = "True signal",linestyle='--',color='b',alpha=0.5)
plt.scatter(t,y_meas, 
                 label = "Baseline RMSE ={:0.3f}".format(np.sqrt(mean_squared_error(y_true,y_meas))),
                   s=80, facecolors='none', edgecolors='r',alpha=0.5)
plt.plot(t,est_y, 
                 label = "EKF estimate RMSE ={:0.3f}".format(np.sqrt(mean_squared_error(y_true,est_y)))
                ,linestyle='-',color='r',alpha=0.5)
plt.legend()
plt.show()

In [ ]:
#RMSE
np.sqrt(mean_squared_error(y_true,y_meas))

In [ ]:
filterPy = True
#same using filterpy
def HJacobian_at(x):
    """ compute Jacobian of H matrix at x """
    return np.array([[dh(x)]])

est2_y = np.zeros(n)
if filterPy:
    Q = np.array([[q_sd**2]])
    R = np.array([[r_sd**2]])
    me = np.array([[f(0)]])
    F = np.array([[df(me)]])
    H = np.array([[dh(me)]])
    P = np.array([[r_sd**2]])
    ekf = ExtendedKalmanFilter (dim_x=1, dim_z=1)
    y_k = np.array(y_meas).flatten().reshape(n,1)
    ekf.x = me
    ekf.F = F
    ekf.H = H
    ekf.P = P
    ekf.Q = Q
    ekf.R = R
    m_array = np.zeros((n,1))
    for k in range(n):
          ekf.predict()
          ekf.update(y_k[k],HJacobian_at,h)
          m_array[k] = ekf.x
    est2_y = m_array.flatten()

In [ ]:
plt.figure(figsize=(20,10))
plt.suptitle("Trajectories and EKF estimates")
plt.plot(t,y_true,label = "True signal",linestyle='--',color='b',alpha=0.5)
plt.scatter(t,y_meas, 
                 label = "Baseline RMSE ={:0.3f}".format(np.sqrt(mean_squared_error(y_true,y_meas))),
                   s=80, facecolors='none', edgecolors='r',alpha=0.5)
plt.plot(t,est2_y, 
                 label = "EKF estimate RMSE ={:0.3f}".format(np.sqrt(mean_squared_error(y_true,est2_y)))
                ,linestyle='-',color='r',alpha=0.5)
plt.legend()
plt.show()

# Exercise 3. (Bearings Only Target Tracking with EKF)

The state of the target at time step $k$ consist of the position $(x_k, y_k)$ and the
velocity $(\dot x_k,\dot y_k)$. The dynamics of the state vector $x_k = (x_k, y_k, \dot x_k, \dot y_k)^T$ are
modeled with the discretized Wiener velocity model:

$$  \begin{pmatrix}
  x_k  \\
  y_k \\
 \dot x_k\\
 \dot y_k \\
  \end{pmatrix} = \begin{pmatrix} 
 1 & 0 & \Delta t & 0 \\
 0 & 1 & 0 & \Delta t \\
 0 & 0 & 1 & 0 \\
 0 & 0 & 0 & 1 
  \end{pmatrix}\begin{pmatrix}
  x_{k-1}  \\
  y_{k-1} \\
 \dot x_{k-1}\\
 \dot y_{k-1}
  \end{pmatrix} + q_{k-1}$$

where $q_k$ is a zero mean Gaussian process noise with covariance

$$Q = \begin{pmatrix} 
 q_1^c \Delta t^3/3 & 0 & q_1^c\Delta t^2/2 & 0 \\
 0 & q_2^c \Delta t^3/3 & 0 & q_2^c \Delta t^2 /2\\
 q_1^c \Delta t^2/2 & 0 & q_1^c \Delta t & 0 \\
 0 & q_2^c \Delta t^2/2 & 0 & q_2^c \Delta t 
  \end{pmatrix}$$
  
  
In this scenario the diffusion coefficients are $q_c^1 = q_c^2 = 0.1$ and the sampling period is $\Delta t = 0.1$. The measurement model for sensor $i \in \{1, 2\}$ is the following:

$$\theta_k^i = tan^{-1} \big(\frac{y_k - s_y^i}{x_k - s_x^i}\big) +r_k$$

where $(s_x^i,s_y^i)$ is the position of the sensor $i$ and $r_k \sim N(0, \sigma^2)$ is a Gaussian
measurement noise with standard deviation of $\sigma = 0.05$ radians. At each sampling time, which occurs 10 times per second (i.e., $\Delta t = 0.1$), both of the two sensors produce a measurement.

In the file angle_ex.m (in MyCourses) there is a baseline solution, which
computes estimates of the position from the crossing of the measurements
and estimates the velocity to be always zero. Your task is to implement an
EKF for the problem and compare the results graphically and in RMSE
sense.

**(a)** Implement an EKF for the bearings only target tracking problem, which
uses the non-linear measurement model (5) as its measurement model
function (not the crossings). Hints:
- Use the Matlab function atan2 in the measurement model instead
of atan to directly get an answer in the range $[-\pi,\pi]$.
- The two measurements at each measurement time can be processed
one at a time, that is, you can simply perform two scalar updates
instead of a single two dimensional measurement update.

In [ ]:
#load simulated data.
Theta = sio.loadmat('Theta.mat')['Theta']
Theta = Theta.swapaxes(0,1)
X = sio.loadmat('X.mat')['X']
X = X.swapaxes(0,1)

In [ ]:
steps = Theta.shape[0]
x0 = np.array([[0],[0],[1],[0]])
S1 = np.array([[-1.5],[0.5]])# Position of sensor 1
S2 = np.array([[1],[1]])      # Position of sensor 2


In [ ]:
qc = 0.1
dt = 0.01
sd = 0.05;
#This is the transition matrix
A = np.array([[1,0,dt,0],
             [0,1,0,dt],
             [0,0,1,0],
             [0,0,0,1]])
#This is the process noise covariance
Q = np.array([[qc*dt**3/3,0, qc*dt**2/2, 0],
       [0, qc*dt**3/3, 0, qc*dt**2/2],
       [qc*dt**2/2, 0, qc*dt, 0],
       [0, qc*dt**2/2, 0, qc*dt]])

In [ ]:
# Initialize to true value
m1 = x0
EST1 = np.zeros((steps,4))
  

# Loop through steps
for k in range(steps):
    # Compute crossing of the measurements
    dx1 = np.cos(Theta[k,0])
    dy1 = np.sin(Theta[k,0])
    dx2 = np.cos(Theta[k,1])
    dy2 = np.sin(Theta[k,1])
    d = np.linalg.solve(np.array([[dx1, dx2],[dy1, dy2]]),np.array([S2[0]-S1[0],S2[1]-S1[1]]))
    # Crossing
    cross_xy = S1 + np.array([[dx1],[dy1]])*d[0]
    # compute estimate
    m1 = np.concatenate([cross_xy.flatten(),[0],[0]])
    EST1[k,:] = m1  

In [ ]:
def rmse(X1,X2):
    return np.sqrt(np.mean(np.sum((X1-X2)**2,axis=1)))

# Baseline solution

Animation

In [ ]:
from matplotlib import animation, rc

In [ ]:
#RMSE
rmse1 = rmse(X,EST1)
rmse1

In [ ]:
%%capture
rc('animation', embed_limit=41030228 )
fig1,ax = plt.subplots(figsize=(20,10))
ax.set_title('Baseline solution')
ax.set_xlim([-2,2])
ax.set_ylim([-2.5,1.5])
ax.plot(X[:,0],X[:,1],linestyle = '--',color='b',alpha=0.5,label='True trajectory')
ax.scatter(S1[0],S1[1],marker = 'X',s=80, facecolors='none', edgecolors='k',label='Sensor 1')
ax.scatter(S2[0],S2[1],s=80, facecolors='none', edgecolors='k',label='Sensor 2')
l, = ax.plot([],[],linestyle = '-',color='r',alpha=0.5,label='Baseline estimate RMSE = {:0.3f}'.format(rmse1))
o, = ax.plot([],[],marker = 'o', color='k')
s_1, = ax.plot([],[],linestyle = '--',color='k')
s_2, = ax.plot([],[],linestyle = '--',color='k')
ax.legend()

def animate(i):
    l.set_data(EST1[:i+1,0],EST1[:i+1,1])
    o.set_data(EST1[i:i+1,0],EST1[i:i+1,1])
    len = 4 
    dx1 = len*np.cos(Theta[i,0]);
    dy1 = len*np.sin(Theta[i,0]);
    dx2 = len*np.cos(Theta[i,1]);
    dy2 = len*np.sin(Theta[i,1]);
    s_1.set_data([S1[0],S1[0]+dx1],[S1[1],S1[1]+dy1])
    s_2.set_data([S2[0],S2[0]+dx2],[S2[1],S2[1]+dy2])
    return (l,o,s_1,s_2)
def init():
    l.set_data([],[])
    o.set_data([],[])
    s_1.set_data([],[])
    s_2.set_data([],[])
    return (l,o,s_1,s_2)
    
ani = animation.FuncAnimation(fig1, animate, init_func=init,frames=len(EST1))

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Baseline solution')
plt.xlim([-2,2])
plt.ylim([-2.5,1.5])
plt.plot(X[:,0],X[:,1],linestyle = '--',color='b',alpha=0.5,label='True trajectory')
plt.plot(EST1[:,0],EST1[:,1],linestyle = '-',color='r',alpha=0.5,label='Baseline estimate RMSE = {:0.3f}'.format(rmse1))
plt.scatter(S1[0],S1[1],marker = 'X',s=80, facecolors='none', edgecolors='k',label='Sensor 1')
plt.scatter(S2[0],S2[1],s=80, facecolors='none', edgecolors='k',label='Sensor 2')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

# EKF



 $$\frac{d arctan(x)}{dx} = \frac{1}{1+x^2}$$
 
 $$ \frac{f(g(x))}{dx} = \frac{df(g(x))}{dx}\frac{g(x)}{dx}$$

In [ ]:
def darctan(x):
    return 1.0/(1+x**2)

def g(mk,s):
    return (mk[1,0]-s[1,0])/(mk[0,0]-s[0,0])

def derivative_x(mk,s):
    return (s[1,0]-mk[1,0])/(s[0,0]**2 - 2*s[0,0]*mk[0,0]+s[1,0]**2 - 2 *s[1,0]*mk[1,0] + mk[0,0]**2 + mk[1,0]**2)

def derivative_y(mk,s):
    return (mk[0,0]-s[0,0])/(s[0,0]**2 - 2.0*s[0,0]*mk[0,0]+s[1,0]**2 - 2 *s[1,0]*mk[1,0] + mk[0,0]**2 + mk[1,0]**2)
    
def dgy(mk,s):
    return 1.0/(mk[0,0]-s[0,0])
def dgx(mk,s):
    return -(mk[1,0]-s[1,0])/(mk[0,0]-s[0,0])**2
def h(mk,s):
    return np.arctan2(mk[1,0]-s[1,0],mk[0,0]-s[0,0])

In [ ]:
m2 = x0           # Initialize to true value
P2 = np.eye(4)        # Some uncertainty
R  = sd**2*np.eye(2)   # The joint covariance
EST2 = np.zeros((steps,4))

for k in range(steps):
    #predict
    mk_ = A @ m2
    Pk_ = A @ P2 @A.T + Q
    #update
    v_k = (Theta[k] - np.array([[h(mk_,S1),h(mk_,S2)]])).T
    #H  = np.array([[darctan(g(mk_,S1))*dgx(mk_,S1),darctan(g(mk_,S1))*dgy(mk_,S1),0,0],
    #               [darctan(g(mk_,S2))*dgx(mk_,S2),darctan(g(mk_,S2))*dgy(mk_,S2),0,0]])
    H = np.array([[derivative_x(mk_,S1), derivative_y(mk_,S1),0,0],
                 [derivative_x(mk_,S2),derivative_y(mk_,S2),0,0]])
    S_k = H@Pk_@H.T + R
    K_k = Pk_@H.T @ np.linalg.inv(S_k)
    m2 = mk_ + K_k @ v_k
    P2 = Pk_ - K_k @ S_k @ K_k.T
    EST2[k,:] = m2.flatten()

In [ ]:
#RMSE 0.4408
rmse2 = rmse(X,EST2)
rmse2

In [ ]:
%%capture
rc('animation', embed_limit=41030228 )
fig1,ax = plt.subplots(figsize=(20,10))
ax.set_title('Baseline solution')
ax.set_xlim([-2,2])
ax.set_ylim([-2.5,1.5])
ax.plot(X[:,0],X[:,1],linestyle = '--',color='b',alpha=0.5,label='True trajectory')
ax.scatter(S1[0],S1[1],marker = 'X',s=80, facecolors='none', edgecolors='k',label='Sensor 1')
ax.scatter(S2[0],S2[1],s=80, facecolors='none', edgecolors='k',label='Sensor 2')
l, = ax.plot([],[],linestyle = '-',color='r',alpha=0.5,label='EKF estimate RMSE = {:0.3f}'.format(rmse2))
o, = ax.plot([],[],marker = 'o', color='k')
s_1, = ax.plot([],[],linestyle = '--',color='k')
s_2, = ax.plot([],[],linestyle = '--',color='k')
ax.legend()

def animate(i):
    l.set_data(EST2[:i+1,0],EST2[:i+1,1])
    o.set_data(EST2[i:i+1,0],EST2[i:i+1,1])
    len = 4 
    dx1 = len*np.cos(Theta[i,0]);
    dy1 = len*np.sin(Theta[i,0]);
    dx2 = len*np.cos(Theta[i,1]);
    dy2 = len*np.sin(Theta[i,1]);
    s_1.set_data([S1[0],S1[0]+dx1],[S1[1],S1[1]+dy1])
    s_2.set_data([S2[0],S2[0]+dx2],[S2[1],S2[1]+dy2])
    return (l,o,s_1,s_2)
def init():
    l.set_data([],[])
    o.set_data([],[])
    s_1.set_data([],[])
    s_2.set_data([],[])
    return (l,o,s_1,s_2)
    
ani = animation.FuncAnimation(fig1, animate, init_func=init,frames=len(EST1))

In [ ]:
from IPython.display import HTML
HTML(ani.to_jshtml())

In [ ]:
plt.figure(figsize=(20,10))
plt.title('EKF solution')
plt.xlim([-2,2])
plt.ylim([-2.5,1.5])
plt.plot(X[:,0],X[:,1],linestyle = '--',color='b',alpha=0.5,label='True trajectory')
plt.plot(EST2[:,0],EST2[:,1],linestyle = '-',color='r',alpha=0.5,label='EKF estimate RMSE = {:0.3f}'.format(rmse2))
plt.scatter(S1[0],S1[1],marker = 'X',s=80, facecolors='none', edgecolors='k',label='Sensor 1')
plt.scatter(S2[0],S2[1],s=80, facecolors='none', edgecolors='k',label='Sensor 2')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

## Same using filterpy

In [ ]:
ekf = ExtendedKalmanFilter (dim_x=4, dim_z=2)
y_k = np.array(Theta).flatten().reshape(steps,2)
ekf.x = x0
ekf.F = A
ekf.P = P2
ekf.Q = Q
ekf.R = R

m_array = np.zeros((steps,4))
def HJacobian_at(x):
    """ compute Jacobian of H matrix at x """
    return np.array([[derivative_x(x,S1), derivative_y(x,S1),0,0],
                 [derivative_x(x,S2),derivative_y(x,S2),0,0]])
def h_x(x):
    return np.array([[h(x,S1)],[h(x,S2)]])

for k in range(steps):
      ekf.predict()
      ekf.update(y_k[k].reshape(2,1),HJacobian_at,h_x)
      m_array[k] = ekf.x.flatten()
est2_y = m_array

In [ ]:
#RMSE 0.4408
rmse2 = rmse(X,est2_y)
rmse2

In [ ]:
plt.figure(figsize=(20,10))
plt.title('EKF solution')
plt.xlim([-2,2])
plt.ylim([-2.5,1.5])
plt.plot(X[:,0],X[:,1],linestyle = '--',color='b',alpha=0.5,label='True trajectory')
plt.plot(est2_y[:,0],est2_y[:,1],linestyle = '-',color='r',alpha=0.5,label='EKF estimate RMSE = {:0.3f}'.format(rmse2))
plt.scatter(S1[0],S1[1],marker = 'X',s=80, facecolors='none', edgecolors='k',label='Sensor 1')
plt.scatter(S2[0],S2[1],s=80, facecolors='none', edgecolors='k',label='Sensor 2')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()